<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/master/deep_NN_with_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explore conv layers and neuron groups

What information we can find in convolutional layers and neuron groups about human preferences?

## Imports

In [0]:
%tensorflow_version 1.x

!git clone https://github.com/arunraja-hub/Preference_Extraction.git

!pip install tf-agents==0.3.0
!pip uninstall tensorflow-probability -y
!pip install tensorflow-probability==0.7.0

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

TensorFlow 1.x selected.
Cloning into 'Preference_Extraction'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (614/614), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 614 (delta 65), reused 511 (delta 10), pack-reused 0
Receiving objects: 100% (614/614), 21.24 MiB | 9.27 MiB/s, done.
Resolving deltas: 100% (65/65), done.
     |████████████████████████████████| 839kB 2.8MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
  Found existing installation: gin-config 0.3.0
    Uninstalling gin-config-0.3.0:
      Successfully uninstalled gin-config-0.3.0
Uninstalling tensorflow-probability-0.7.0:
  Successfully uninstalled tensorflow-probability-0.7.0
     |████████████████████████████████| 983kB 2.7MB/s 
  Found existing installation: tensorflow-probability 0.10.0rc0
    Uninstalling tensorflow-probability-0.10.0rc0:
      Successfully uninstalled tensorflow-probability-0.10.0rc0


In [0]:
import numpy as np

from tf_agents.trajectories.time_step import TimeStep
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import BoundedTensorSpec
from tf_agents.networks import q_network

import lucid.modelzoo.vision_models as models

import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import numpy as np
import io
import collections

import urllib.request
from urllib.error import HTTPError

In [0]:
from lucid.misc.channel_reducer import ChannelReducer

## Read Data

In [0]:
class Trajectory(
    collections.namedtuple('Trajectory', [
        'step_type',
        'observation',
        'action',
        'policy_info',
        'next_step_type',
        'reward',
        'discount',
    ])):
  """Stores the observation the agent saw and the action it took.
      The rest of the attributes aren't used in this code."""
  __slots__ = ()

class ListWrapper(object):
  def __init__(self, list_to_wrap):
    self._list = list_to_wrap

  def as_list(self):
    return self._list

class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
      if name == "Trajectory":
        return Trajectory
      if name == "ListWrapper":
        return ListWrapper

      return super(RenameUnpickler, self).find_class(module, name)

def rename_load(s):
    """Helper function analogous to pickle.loads()."""
    return RenameUnpickler(s, encoding='latin1').load()

In [0]:
# Modified read trajectories functions to read files from local storage

def load_file(full_path):
    try:
        with open(full_path, 'rb') as f:
            data = rename_load(f)
            return data
    except:
        return None
    
def all_load_data(base_path):
    
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
    
    futures = []
    for i in range(5000):
        full_path = os.path.join(base_path, "ts"+str(i)+".pickle")
        future = executor.submit(load_file, full_path)
        futures.append(future)
    
    raw_data = []
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            raw_data.append(result)
    
    return raw_data


all_raw_data = all_load_data("Preference_Extraction/data/simple_env_1/")

In [0]:
assert len(all_raw_data) == 475

In [0]:
activations = []
observations = []
preferences = []

In [0]:
# Load up data into arrays
for data in all_raw_data:
    for i in range(data.observation.shape[0]):
        observations.append(np.copy(data.observation[i]))
        activations.append(np.copy(data.policy_info["activations"][i]))
        preferences.append(data.policy_info['satisfaction'].as_list()[i])

observations = np.array(observations)
activations = np.array(activations)
preferences = np.array(preferences)

## Load model get conv activations

In [0]:
cpt_name = "Preference_Extraction/model_ckpt"

In [0]:
tf.reset_default_graph()
input_shape = [14, 16, 5]

my_input = tf.placeholder(tf.float32, shape=[None] + input_shape, name="my_input")
q_vals = q_network.QNetwork(input_tensor_spec=TensorSpec(shape=(14, 16, 5)), action_spec=BoundedTensorSpec((), tf.int32, 0, 2), conv_layer_params = [[16, 3, 1], [32, 3, 2]], fc_layer_params = [64])(my_input)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
cpt_var_info = tf.compat.v1.train.list_variables(cpt_name)
cpt_var_info = [var for var in cpt_var_info if (("bias" in var[0]) or ("kernel" in var[0])) and not ("OPTIMIZER" in var[0]) and not ("_target_q_network" in var[0])]
shape_to_cpt_var_name = {tuple(var[1]): var[0] for var in cpt_var_info}

current_vars = tf.get_collection(tf.GraphKeys.VARIABLES)
shape_to_current_var_name = {tuple(var.get_shape().as_list()): var.name[:-2] for var in current_vars}

var_name_to_prev_var_name = {}
for shape in shape_to_current_var_name:
  var_name_to_prev_var_name[shape_to_current_var_name[shape]] = shape_to_cpt_var_name[shape]

Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


In [0]:
tf.train.warm_start(cpt_name, var_name_to_prev_var_name=var_name_to_prev_var_name)
init_op = tf.global_variables_initializer()

INFO:tensorflow:Warm-starting from: Preference_Extraction/model_ckpt


INFO:tensorflow:Warm-starting from: Preference_Extraction/model_ckpt


INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.


INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.


INFO:tensorflow:Warm-started 8 variables.


INFO:tensorflow:Warm-started 8 variables.


In [0]:
conv_activations = []
conv_activation_tensor = tf.get_default_graph().get_tensor_by_name(
    "QNetwork/EncodingNetwork/EncodingNetwork/conv2d/Relu:0")

with tf.Session() as sess:
  sess.run(init_op)

  for img in observations:
    conv_act = sess.run(conv_activation_tensor, {my_input: np.array([img])})[0]
    conv_activations.append(conv_act)

conv_activations = np.array(conv_activations)

## Data Recap

Now we should have
* preferences
* dense layer activations
* conv layer activations
* observations

In [0]:
print('Obs.shape', observations.shape)
print('ConvAct.shape', conv_activations.shape)
print('DensAct.shape', activations.shape)
print('Pref.shape', preferences.shape)

Obs.shape (23750, 14, 16, 5)
ConvAct.shape (23750, 12, 14, 16)
DensAct.shape (23750, 64)
Pref.shape (23750,)


In [0]:
import joblib

In [0]:
joblib.dump(observations, 'obs.pkl')
joblib.dump(conv_activations, 'convAct.pkl')
joblib.dump(activations, 'densAct.pkl')
joblib.dump(preferences, 'pref.pkl')

['pref.pkl']

## Dimensionality reduction (unused)

Let us try to compress convolutional activations using the neuron groups techniques used. Perhaps this can be applied to observations too?

In [0]:
def neuron_groups(acts, n_groups = 4):
    nmf = ChannelReducer(n_groups, "NMF")
    spatial_factors = nmf.fit_transform(acts)[0].astype("float32")
    channel_factors = nmf._reducer.components_.astype("float32")
    return spatial_factors, channel_factors

In [0]:
conv_activations_space = []
conv_activations_chan = []

for c_act in conv_activations:
    c_act_s, c_act_ch = neuron_groups(c_act)
    conv_activations_space.append(c_act_s)
    conv_activations_chan.append(c_act_ch)

conv_activations_space = np.array(conv_activations_space)
conv_activations_chan = np.array(conv_activations_chan)

KeyboardInterrupt: ignored

In [0]:
print('ConvAct.shape', conv_activations.shape)
print('ConvAct_spatial.shape', conv_activations_space.shape)
print('ConvAct_channels.shape', conv_activations_chan.shape)

ConvAct.shape (23750, 12, 14, 16)
ConvAct_spatial.shape (23750, 14, 4)
ConvAct_channels.shape (23750, 4, 16)


In [0]:
ys = preferences
xs = np.concatenate((
    conv_activations_space.reshape(conv_activations_space.shape[0], -1),
    conv_activations_chan.reshape(conv_activations_chan.shape[0], -1)
), axis=1)

## Modelling (restart kernel)

In [0]:
import joblib
import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import scipy
from scipy import ndimage
from sklearn import metrics
from sklearn.utils import shuffle
import io
import collections

In [0]:
observations = joblib.load('obs.pkl')
conv_activations = joblib.load('convAct.pkl')
activations = joblib.load('densAct.pkl')
preferences = joblib.load('pref.pkl')

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

#Data structure analysis (can be skipped)

In [0]:
#print("Convolutional activations:")
#print(conv_activations)
#print("Activations:")
#print(activations)

test_xs = []

#for conv_activation in conv_activations:
  #print(conv_activation)

#for activation in activations:
  #print(activation)

for conv_activation, activation in zip(conv_activations, activations):
  test_xs.append([conv_activation, activation])

test_xs = np.array(test_xs)
print("conv_activations[0] = ")
print(conv_activations[0])
print("activations[0] = ")
print(activations[0])
print("test_xs[0] = ")
print(test_xs[0])
print(test_xs.shape)

conv_activations[0] = 
[[[0.         0.         0.66749555 ... 0.         0.         1.4185064 ]
  [0.         0.         0.         ... 0.         0.         0.93666255]
  [0.         0.         0.         ... 0.         0.         0.8227383 ]
  ...
  [0.         0.         0.         ... 0.         0.         1.1279838 ]
  [0.         0.         0.         ... 0.         0.         0.9015943 ]
  [0.         0.         0.         ... 0.         0.         1.1268153 ]]

 [[0.         0.         0.         ... 0.         0.         1.120673  ]
  [0.         0.         0.         ... 0.         0.         0.89428335]
  [0.         0.         0.         ... 0.         0.         1.0960746 ]
  ...
  [0.         0.         0.         ... 0.         0.         0.8928306 ]
  [0.         0.         0.         ... 0.         0.         1.1180516 ]
  [0.         0.         0.         ... 0.         0.         0.8916621 ]]

 [[0.         0.         0.         ... 0.         0.         0.8855195 ]

## Training

In [0]:
xs_raw = []
xs = []

for conv_activation, activation in zip(conv_activations, activations):
  xs_raw.append([conv_activation, [[activation]]])

for element in xs_raw:
  temp = []
  for subelement in element:
    for subsubelement in subelement:
      for subsubsubelement in subsubelement:
        for subsubsubsubelement in subsubsubelement:
          temp.append(subsubsubsubelement)
  temp = np.array(temp)
  xs.append(temp)
xs = np.array(xs)
ys = (preferences > -6).astype(int)

In [0]:
def get_val_auc(logs):
      for key in logs:
        if key.startswith('val_auc'):
          return logs[key]

class BestStats(tf.keras.callbacks.Callback):
  """A callback to keep track of the best val accuracy and auc seen so far."""
  def on_train_begin(self, logs):
      self.bestMetric = -float('inf')
      self.bestLogs = None
      self.bestTrain = -float('inf')
      self.num_epochs = 0

  def on_epoch_end(self, epoch, logs):
    self.num_epochs += 1
    self.bestTrain = max(self.bestTrain, logs.get('accuracy'))

    val_accuracy = logs.get('val_accuracy')
    if val_accuracy == None:
      return 

    val_auc = get_val_auc(logs)
    
    metric = (val_accuracy + val_auc) / 2.0

    if metric > self.bestMetric:
      self.bestMetric = metric
      self.bestLogs = logs

In [0]:
xs.shape

(23750, 2752)

In [0]:
# try a different model that doesn't use convolutions and Flattens() the input at first

def get_model(reg_amount, drop_rate, reduction_alg, n_components):
  del reduction_alg, n_components

  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dropout(drop_rate),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(reg_amount)),
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(.01),
                loss='binary_crossentropy',
                metrics=['accuracy',
                        tf.keras.metrics.AUC()
                        ],
                )
  return model


In [0]:
use_activations = False

all_hparam_possibilities = [
    {"drop_rate": [.0,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None]},
    {"drop_rate": [.2,], "reg_amount": [.0], 'reduction_alg': [None], 'n_components': [None]}]

In [0]:
num_train = 50
num_val = 1000
epochs = 400
print("use_activations:", use_activations, "num_train:", num_train, "epochs", epochs)
if num_train > 50:
  print("More than 50 train data!!!!!!!!")

# each item in all_hparam_possibilities specifies valid hyper params to try. Put parameters that don't make sense together in separate lists.

hparam_combinations = []
for hparam_possibilities in all_hparam_possibilities:
  hparam_keys, hparam_values = zip(*hparam_possibilities.items())
  hparam_combinations.extend([dict(zip(hparam_keys, v)) for v in itertools.product(*hparam_values)])
random.shuffle(hparam_combinations)
print("len(hparam_combinations)", len(hparam_combinations), "hparam_combinations", hparam_combinations)

def modify_x_for_reduce(xs):
  reshaped_x = np.reshape(xs, [xs.shape[0], -1])
  # Make everything positive because some reductions don't work with negatives.
  reshaped_x -= np.min(reshaped_x)
  return reshaped_x

def unsup_exstract(xs, reg_amount, drop_rate, layer_sizes, reduction_alg, n_components):
  del reg_amount, drop_rate, layer_sizes

  print("Using unsupervised feature extraction.")

  dim_reduct_model = ChannelReducer(reduction_alg=reduction_alg, n_components=n_components)
  xs = dim_reduct_model.fit_transform(modify_x_for_reduce(xs))
  return xs

def train_best_logs(xs, ys, num_val, do_summary, hparams, get_model):
  """Trains the model and retruns the logs of the best epoch. randomly splits the train and val data before training."""
  tf.keras.backend.clear_session()
  model = get_model(**hparams)
  xs, ys = shuffle(xs, ys)

  xs_val = xs[num_train:num_train+num_val]
  ys_val = ys[num_train:num_train+num_val]
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=0)
  best_stats = BestStats()
  model.fit(xs[:num_train], ys[:num_train], epochs=epochs, batch_size=256, validation_freq=1, callbacks=[best_stats, early_stopping], validation_data=(xs_val, ys_val), verbose=0)
  if do_summary:
    model.summary()
    print("best train accuracy:", best_stats.bestTrain)
    print("Number of epochs:", best_stats.num_epochs)
  return best_stats.bestLogs

def multiple_train_ave(hparams):
  """Trains the model multiple times with the same parameters and returns the average metrics"""
  start = time.time()
  all_val_auc = []
  all_val_accuracy = []

  if hparams['reduction_alg'] != None:
    xs_for_train = unsup_exstract(xs, **hparams)
  else:
    xs_for_train = xs

  do_summary = True
  for i in range(5):
    logs = train_best_logs(xs_for_train, ys, num_val, do_summary, hparams, get_model)
    all_val_auc.append(get_val_auc(logs))
    all_val_accuracy.append(logs.get('val_accuracy'))
    do_summary = False 

  mean_val_auc = np.mean(all_val_auc)
  mean_val_accuracy = np.mean(all_val_accuracy)
  metric = (mean_val_auc + mean_val_accuracy) / 2.0
  print_data = ("mean_val_auc", mean_val_auc, "mean_val_accuracy", mean_val_accuracy, "metric", metric, "val_auc_std", np.std(all_val_auc), "val_accuracy_std", np.std(all_val_accuracy))

  end = time.time()
  print("Seconds per hyperparam config", end - start)
  # GPU: ('Seconds per hyperparam config', 16.970870971679688)

  return metric, print_data

best_metric = -float('inf')

run_num = 0
for hparams in hparam_combinations:
  print("hparams", hparams)

  metric, print_data = multiple_train_ave(hparams)

  print(print_data)
  if metric > best_metric:
    best_metric = metric
    best_print_data = print_data
    best_hparams = hparams

  run_num += 1
  print("fract done", run_num/float(len(hparam_combinations)))
  print
  print("==============================================================================================")
  print
  sys.stdout.flush()

print("best_hparams", best_hparams)
print("best results", best_print_data)
print("Retraining on the best_hparams to make sure we didn't just get good results by random chance.")

_, print_data = multiple_train_ave(best_hparams)
print("Result of retrain on the best hyperparameters", print_data)

use_activations: False num_train: 50 epochs 400
len(hparam_combinations) 2 hparam_combinations [{'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None}, {'drop_rate': 0.2, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None}]
hparams {'drop_rate': 0.0, 'reg_amount': 0.0, 'reduction_alg': None, 'n_components': None}
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  704768    
_________________________________________________________________
dense_1 (Dense)              multiple                  131584    
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_______________________________